# Data Mining For Materials Science from published litterature

Here goes some text describing the project

In [19]:
import requests
from xml.etree import ElementTree as ET
import pandas as pd
import json
from semanticscholar import SemanticScholar

In [2]:
APIKey = '<Include your Elsevier API Key here>' ## Elsevier API Key

In [3]:
doi_list = pd.read_csv('../data/Saeki_papers_doi.csv',names=['paper_id', 'doi', 'source'])
doi_list.head(5)

,paper_id,doi,source
0,1,10.1063/1.2899996,American Institute of Physics (AIP)
1,2,10.1002/adma.200602496,Wiley
2,3,10.1021/ja710079w,American Chemical Society Publications (ACS)
3,4,10.1038/ncomms6293,Nature
4,5,10.1038/NPHOTON.2009.192,Nature


In [4]:
elsevier_dois = {}
for i in range(doi_list.shape[0]):
    doi = doi_list.loc[i]
    if doi['source'] == 'Elsevier':
        elsevier_dois[doi['paper_id']] = doi['doi']

#print(elsevier_dois)
len(elsevier_dois)

47

## download figures and save them

In [5]:
paper_folder = './elsevier_papers/'
image_folder = './elsevier_images/'

object_tag = '{http://www.elsevier.com/xml/svapi/article/dtd}object'
jpeg_id_attrib = 'multimediatype'
jpeg_id_value = 'JPEG image file'

In [6]:
for paper_id, doi in elsevier_dois.items():
    # download paper 
    paper_name = paper_folder + paper_id + '.xml'
    paper_url = "https://api.elsevier.com/content/article/doi/" + doi + '?' + APIKey
    data = requests.get(paper_url)
    paper = ET.ElementTree(ET.fromstring(data.content))
    paper.write(paper_name)
    
    # find image and download
    object_lists = paper.iter(object_tag)
    image_name_prefix = image_folder + paper_id + '_'
    for obj in object_lists:
        if obj.attrib[jpeg_id_attrib] == jpeg_id_value:
            jpeg_name = image_name_prefix + obj.attrib['ref'] + '.jpeg'
            jpeg_url = obj.text + '&' + APIKey
            jpeg = requests.get(jpeg_url)
            with open(jpeg_name, 'wb') as f:
                f.write(jpeg.content)
         

## do search with elsevier api

In [7]:
search_key_words  = ['OPV'] 
total_number_of_results =  20000
number_of_outputs_per_page = 25
sorting = 'relevancy' #,pubyear'

search_qs = '+AND+'.join(search_key_words)
print(search_qs)
search_url = f'https://api.elsevier.com/content/search/scopus?APIKey={APIKey}&query=KEY({search_qs})&count={number_of_outputs_per_page}&sort={sorting}'

search_result_entries = []

totalPage = int(total_number_of_results / number_of_outputs_per_page)
nextpage = search_url

search_result_entries = []

for pageNum in range(totalPage):
    
    # skip even pages 
    if (pageNum + 1) % 2 == 0: continue
        
    # skip odd pages
    #if pageNum % 2 == 1: continue
    
    print(f"Query Page: {pageNum + 1}")
    query_url = search_url + '&start=' + str(number_of_outputs_per_page * pageNum)
    
    # Call the search API
    search_response = requests.get(query_url)
    search_response_content = json.loads(search_response.content)

    # Append the results from that page to our results set
    search_result_entries += search_response_content['search-results']['entry']

    
#date = '2011-2021'
#search_url = search_url + '&date=' + date

# start = 20
# search_url = search_url + '&start=' + str(start)

#search_url

OPV
Query Page: 1
Query Page: 3
Query Page: 5
Query Page: 7
Query Page: 9
Query Page: 11
Query Page: 13
Query Page: 15
Query Page: 17
Query Page: 19
Query Page: 21
Query Page: 23
Query Page: 25
Query Page: 27
Query Page: 29
Query Page: 31
Query Page: 33
Query Page: 35
Query Page: 37
Query Page: 39
Query Page: 41
Query Page: 43
Query Page: 45
Query Page: 47


KeyError: 'entry'

In [5]:
search_key_words  = ['OPV'] 
number_of_outputs = 25
sorting = 'relevancy' #,pubyear'

search_qs = '+AND+'.join(search_key_words)
print(search_qs)
search_url = f'https://api.elsevier.com/content/search/scopus?APIKey={APIKey}&query=KEY({search_qs})&count={number_of_outputs}&sort={sorting}'

search_result_entries = []

pageNum = 0

nextpage = search_url
while nextpage != None:
    pageNum = pageNum + 1
    print(f"Page: {pageNum}")

    # Temporary limit to get only the first 3 pages.
    if pageNum > 3:
        break

    # Call the search API
    search_response = requests.get(nextpage)
    search_response_content = json.loads(search_response.content)

    # Append the results from that page to our results set
    search_result_entries.append(search_response_content['search-results']['entry'])

    # Check if there's a next page
    nextpage = None
    results_links = search_response_content['search-results']['link']
    for link in results_links:
        if link['@ref']== 'next':
            nextpage = link['@href']
    

OPV
Page: 1
Page: 2


KeyError: 'search-results'

In [6]:
search_response_content

{'service-error': {'status': {'statusCode': 'AUTHENTICATION_ERROR',
   'statusText': 'Invalid API Key'}}}

In [30]:
def parse_results_from_scopus_search(search_result, APIKey):
    
    if 'prism:doi' not in search_result.keys(): return None
    doi         = search_result.get('prism:doi', 'N/A')
    title       = search_result.get('dc:title', 'N/A')
    year        = search_result.get('prism:coverDate', 'N/A').split('-')[0]
    
    #scopus_url  = search_result['prism:url'] + '?APIKey=' + APIKey
    #headers = {'accept':'application/json'}
    #details = requests.get(scopus_url,headers=headers)
    #details_json = json.loads(details.text)['abstracts-retrieval-response']['coredata']
        
    #publisher   = details_json.get('dc:publisher', 'N/A')
    #publication = details_json.get('prism:publicationName', 'N/A')
    publisher = ""
    publication = ""

    return pd.DataFrame([[doi, publication, publisher, title, year]], columns=columns)

In [31]:
columns = ['doi', 'publication', 'publisher', 'title', 'year']
final_df = pd.DataFrame(columns=columns)

for result in search_result_entries:
    df = parse_results_from_scopus_search(result, APIKey)
    #print(df)
    if df is not None:
        final_df = pd.concat([final_df, df], ignore_index = True)

In [32]:
print(final_df.shape)
print(final_df.head(10))

(515, 5)
                               doi                                publication  \
0       10.1186/s12859-021-04045-3                         BMC Bioinformatics   
1  10.1016/j.commatsci.2021.110599            Computational Materials Science   
2     10.1016/j.jlumin.2021.118192                    Journal of Luminescence   
3      10.1016/j.ijleo.2021.166937                                      Optik   
4      10.1016/j.jvacx.2021.100102                                 Vaccine: X   
5       10.1007/s11664-021-09020-5            Journal of Electronic Materials   
6       10.1016/j.xcrp.2021.100498              Cell Reports Physical Science   
7           10.1002/anie.202102622  Angewandte Chemie - International Edition   
8           10.1021/acsami.1c08487       ACS Applied Materials and Interfaces   
9          10.3390/vaccines9070688                                   Vaccines   

                   publisher  \
0         BioMed Central Ltd   
1              Elsevier B.V.   
2  

In [34]:
#save search_results
final_df_name = 'scopus_search_results_odd_pages_before_page46.csv'
final_df.to_csv(final_df_name)

## Now that we have search results, let's retrieve the abstracts

In [9]:
final_df = pd.read_csv('../data/search_results_photovoltaicANDdonor_with_abstract.csv')

In [33]:
# retrieves a paper from the semantic scholar database given its {paperid}. 
# attempts it up to {retries} times and each attempt times out after {timeout}
sch = SemanticScholar()
def force_get_paper(paperid, retries = 3, timeout = 15):
    paperid = str(paperid)
    if retries == 0:
        return None
    try:
        return sch.get_paper(paperid, timeout=timeout)
    except:
        print(f'timeout: retrying request for paper id {paperid}. retries: {retries}')
        return force_get_paper(paperid, retries = retries-1, timeout = timeout)

In [ ]:
abstracts = []
for paperid in final_df['doi']:
    paper_details = force_get_paper(paperid)
    try:
        abstract = paper_details['abstract']
    except:
        abstract = ""
    abstracts.append(abstract)

final_df['Abstract'] = abstracts